This Script check the missing data imputation methods for Train envirnmental.
Interpolation methods, KNN, and micefirest are considered for analysis. 
In some columns, there are missing values for almost 40 weeks after baseline.

In [ ]:
https://ishanjainoffical.medium.com/missing-value-treatment-advanced-methods-f7fa05ec0f39

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import miceforest as mf
import random
import sklearn.neighbors._base 
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

In [2]:
# Function to introduce missing values randomly (modified from previous example)
def create_missing_values(data, missing_prob):
  for col in data.columns:
    missing_indices = np.random.choice(data.index, size=int(missing_prob * len(data)), replace=False)
    data.loc[missing_indices, col] = np.nan
  return data

def fill_missing_with_mean(group):
    return group.fillna(group.mean())

In [3]:
path_env = "C:/Users/Mahin Vazifehdan/Desktop/Datasets/BrainTeaser/CLEF_2024/Original_data/train_environmental_meas.csv"
env_data = pd.read_csv(path_env) # 113923 * 36

# Create a mapping dictionary to assign numerical values to patient IDs
unique_patients = env_data['patient_id'].unique()
patient_id_map = {patient_id: idx+1 for idx, patient_id in enumerate(unique_patients)}
env_data["patient_number"] = env_data["patient_id"].map(patient_id_map)

# Make air pollution who percentage features
Env_features = {'PM25': 1,
                'PM10': 2,
                'CO': 3,
                'NO2': 4,
                'O3': 5,
                'SO2': 6}

for key in Env_features:
    three_env_feature = env_data.filter(like=key)
    env_data[key+"_who_percentage"] = three_env_feature.iloc[:,2] / three_env_feature.iloc[:,0]
selected_env_feature = env_data.filter(regex='mean|percentage')
env_data = pd.concat([env_data[["patient_number","week_from_baseline"]], selected_env_feature], axis=1)

In [4]:
completed_env = env_data.dropna()
# Select data after -4 weeks of baseline
completed_env = completed_env[completed_env["week_from_baseline"] >= -4] 
completed_env.to_csv("completed_env_data_-4More.csv", index= False) # 10869 * 17

missing_probability = 0.3
miss_env_data = create_missing_values(completed_env.iloc[:,2:], missing_probability)
miss_env_data = pd.concat([completed_env[["patient_number","week_from_baseline"]], miss_env_data], axis=1)
miss_env_data.to_csv("Train_env_with_missing_0.1_-4More.csv",index= False) # 10869 * 17

In [5]:
# K-means imputation
kmean_completed = completed_data.copy()
kmean_missing = missing_data.copy()

n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmean_completed['Cluster'] = kmeans.fit_predict(kmean_completed) 
kmean_missing['Cluster'] = kmean_completed['Cluster']

for cluster_label in range(n_clusters): 
    cluster_subset = kmean_completed[kmean_completed['Cluster'] == cluster_label]
    cluster_mean = cluster_subset[kmean_completed.columns[11:19]].mean().round()
    filled_data = kmean_missing.loc[kmean_missing['Cluster'] == cluster_label,
                                    kmean_missing.columns[11:19]].fillna(cluster_mean, inplace=False)
    kmean_missing.update(filled_data)
    
kmean_missing.drop(["Cluster"], axis=1, inplace = True)
kmean_missing.head()

NameError: name 'completed_data' is not defined

In [12]:
# Impute with LOCF method
LOCF_imputed = miss_env_data.groupby('patient_number').apply(lambda x: x.fillna(method='ffill'))
LOCF_imputed = LOCF_imputed.groupby('patient_number').apply(fill_missing_with_mean)

#Interpolation Methods
linear_imputed_data = miss_env_data.groupby('patient_number').apply(lambda group: group.interpolate(method='linear'))
linear_imputed = linear_imputed_data.groupby('patient_number').apply(fill_missing_with_mean)

nearest_imputed_data = miss_env_data.groupby('patient_number').apply(lambda group: group.interpolate(method='nearest'))
nearest_imputed = nearest_imputed_data.groupby('patient_number').apply(fill_missing_with_mean)


In [14]:
# Mice forest imputation
mice_missing = miss_env_data.copy()

kernel = mf.ImputationKernel(
  data=mice_missing.iloc[:,2:],
  save_all_iterations=True,
  random_state=1343)

kernel.mice(5,verbose=True)
mice_imputed = kernel.complete_data(dataset=0, inplace=False)
mice_imputed = pd.concat([completed_env[["patient_number","week_from_baseline"]], mice_imputed], axis=1) #10931 * 22

Initialized logger with name mice 1-5
Dataset 0
1  | PM25_mean | NO2_who_percentage | CO_who_percentage | PM10_who_percentage | PM25_who_percentage | max_temperature_mean | min_temperature_mean | average_temperature_mean | precipitation_mean | global_radiation_mean | sealevel_pressure_mean | humidity_mean | wind_speed_mean | SO2_mean | O3_mean | NO2_mean | CO_mean | PM10_mean | O3_who_percentage | SO2_who_percentage
2  | PM25_mean | NO2_who_percentage | CO_who_percentage | PM10_who_percentage | PM25_who_percentage | max_temperature_mean | min_temperature_mean | average_temperature_mean | precipitation_mean | global_radiation_mean | sealevel_pressure_mean | humidity_mean | wind_speed_mean | SO2_mean | O3_mean | NO2_mean | CO_mean | PM10_mean | O3_who_percentage | SO2_who_percentage
3  | PM25_mean | NO2_who_percentage | CO_who_percentage | PM10_who_percentage | PM25_who_percentage | max_temperature_mean | min_temperature_mean | average_temperature_mean | precipitation_mean | global_radia

In [15]:
#KNN imputation K=5
Knn_missing = miss_env_data.copy()
imputer = KNNImputer()
KNN_imputed = imputer.fit_transform(Knn_missing.iloc[:,2:])
P_number_W_baseline = completed_env[["patient_number","week_from_baseline"]]
concatenated_array = np.concatenate((P_number_W_baseline, KNN_imputed), axis=1)
KNN_imputed = pd.DataFrame(concatenated_array, columns = miss_env_data.columns)

patient_number              0
week_from_baseline          0
PM25_mean                   0
PM10_mean                   0
CO_mean                     0
NO2_mean                    0
O3_mean                     0
SO2_mean                    0
wind_speed_mean             0
humidity_mean               0
sealevel_pressure_mean      0
global_radiation_mean       0
precipitation_mean          0
average_temperature_mean    0
min_temperature_mean        0
max_temperature_mean        0
PM25_who_percentage         0
PM10_who_percentage         0
CO_who_percentage           0
NO2_who_percentage          0
O3_who_percentage           0
SO2_who_percentage          0
dtype: int64

In [16]:
# Compare imputed data with real values
# LOCF
mse_locf = mean_squared_error(completed_env.values.flatten(), LOCF_imputed.values.flatten())
# Interpolation
mse_linear = mean_squared_error(completed_env.values.flatten(), linear_imputed.values.flatten())

mse_nearest = mean_squared_error(completed_env.values.flatten(), nearest_imputed.values.flatten())

#Mice
mse_mice = mean_squared_error(completed_env.values.flatten(), mice_imputed.values.flatten())
#KNN
mse_knn = mean_squared_error(completed_env.values.flatten(), KNN_imputed.values.flatten())

print("LOCF Error:", mse_locf.round(4))
print("Interpolation Linear Error:", mse_linear.round(4))
print("Interpolation Hotdeck  Error:", mse_nearest.round(4))
print("MiceForest Error:", mse_mice.round(4))
print("KNN Error:", mse_knn.round(4)) # The best 

LOCF Error: 25.897
Interpolation Linear Error: 16.2652
Interpolation Polynomial Error: 24.8095
Interpolation Hotdeck  Error: 23.4789
Interpolation quadratic Error: 24.8095
Interpolation Rolling Error: 34.9594
MiceForest Error: 12.9738
KNN Error: 6.6869
